In [1]:
## Load CCSum test data (it contains both extractive and abstractive subsets)

from datasets import load_dataset, load_from_disk

dataset = load_dataset("/mnt/ceph_rbd/datasets/ccsum")  # point to the dir of the unzipped folder from dropbox

# Get extractive and abstractive subsets:
 
# abstractive subset of the dataset
dataset_abstractive = dataset.filter(lambda x: x["abstractiveness_bin"] == "high")
print(dataset_abstractive['test'][0])
 
# extractive subset of the dataset
dataset_extractive = dataset.filter(lambda x: x["abstractiveness_bin"] == "low")
print(dataset_extractive['test'][0])

/mnt/ceph_rbd/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 1349911 examples [00:45, 29531.31 examples/s]
Generating validation split: 10000 examples [00:00, 23492.12 examples/s]
Generating test split: 10000 examples [00:00, 19921.64 examples/s]
Filter: 100%|██████████| 10000/10000 [00:00<00:00, 34140.06 examples/s]


{'summary': 'Female employees that sued Google for paying them less than men in equivalent positions have won $118 million in settlement with the tech giant.', 'article': 'As part of the settlement, independent experts will review Google’s hiring practices and pay-equity studies.\nGoogle agreed to pay $118 million to settle a lawsuit claiming the tech giant had discriminated against women in pay and promotions.\nThe settlement concludes a class-action suit in San Francisco Superior Court initiated in 2017 by three female former employees who said that Google, a unit of Alphabet placed them in lower-job levels than similarly qualified males, leading to lower pay, and denied the women promotions or transitions to other teams that would have led to better career advancement.', 'cluster_id': 'a802c600923d4fa78ae7b0f1de4fbedd', 'summary_id': 'b035890b81c32b600f24aa569b06983b4d7952948ccf69ede31d19e9cfcb9c82', 'article_id': 'd016fb54967b7bba7ed46afcfe4983e2584b854d3ead801cd273a6d4c2497125', '

Filter: 100%|██████████| 10000/10000 [00:00<00:00, 36714.11 examples/s]

{'summary': 'A car has driven into a crowd of people in western Berlin, killing one person and smashing through a shop window, police say, in a district of the German capital popular with tourists and shoppers.', 'article': 'BERLIN • A 29-year-old man drove his car into a crowd of people in Berlin yesterday, killing one person as his vehicle veered onto a pavement twice in a district of the German capital popular with tourists and shoppers, police said.\nPolice identified the driver as a German-Armenian man, whose car eventually crashed into a shop window. He was detained by bystanders and handed over to the authorities.\n"It is too early to speculate" about what caused the incident, an Interior Ministry spokesman told a regular federal government news conference in Berlin. Police said it was not yet clear whether it was an accident or a deliberate act.\nGermany\'s Bild newspaper said the person killed was a teacher who had been with a group of her schoolchildren in the street near the

In [4]:
print(dataset_abstractive['test'][103]['article'])
print(dataset_abstractive['test'][103])

Something went wrong, please try again later.
Get the latest news from CroydonCentral straight to your inbox
A former Met officer has been fined £500 following an investigation into a viral social media video that showed him masturbating while on duty. Kevin Phillips, who has since resigned, was a part of the Met's Roads and Transport Policing Command when he was filmed by a member of the public.
The video - filmed in Dog Kennel Hill park in East Dulwich - captured the badge number of former police community support officer Phillips, 56, indicating he was a serving officer. The video was then shared rapidly across social media platforms and was viewed tens of thousands of times. The footage appeared to show a man “committing a sex act on a London park bench in broad daylight”, according to the MailOnline.
Chief Superintendent Simon Ovens, leader of Met’s Roads and Transport Policing Command, said: “Phillips' behaviour that day was repulsive and is contrary to everything the decent offi

In [1]:
import torch
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from datasets import load_dataset

import json
from pathlib import Path
from tqdm import tqdm


input_key = {
    "xsum": "document",
    "cnn_dm": "article",
    "ccsum": "article"
}

output_key = {
    "xsum": "summary",
    "cnn_dm": "highlights",
    "ccsum": "summary"
}

dataset_name = "ccsum"
if dataset_name == "xsum":
    test_data = load_dataset("xsum", split="test")
elif dataset_name == "cnn_dm":
    test_data = load_dataset('cnn_dailymail', '3.0.0', split='test')
elif dataset_name == "ccsum":
    # load CCSum test data (abstractive subset)
    ccsum_dataset = load_dataset("/mnt/ceph_rbd/datasets/ccsum")
    dataset_abstractive = ccsum_dataset.filter(lambda x: x["abstractiveness_bin"] == "high")
    test_data = dataset_abstractive['test']


model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto",
                                             use_auth_token=True,
                                             cache_dir="/mnt/ceph_rbd/llms")
tokenizer = AutoTokenizer.from_pretrained(model_name)

input_token_length = []
for idx, sample in tqdm(enumerate(test_data)):
    doc = sample[input_key[dataset_name]]
    summary = sample[output_key[dataset_name]]

    input_ids = tokenizer(doc, truncation=False, return_tensors="pt").input_ids
    input_token_length.append(input_ids.shape[-1])
    if input_ids.shape[-1] > 2500:
        print(idx)

print("Average number of input tokens: ", sum(input_token_length) / len(input_token_length))
print("Max number of input tokens: ", max(input_token_length))

/mnt/ceph_rbd/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/ceph_rbd/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.69it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
1276it [00:01, 789.66it/s]

1125


4947it [00:06, 804.66it/s]

Average number of input tokens:  383.6521123913483
Max number of input tokens:  2664


In [4]:
# Test whether the gold summary only contains a single sentence
# Note: most of the summaries are one sentence; others may contain two sentences

import re
from tqdm import tqdm

def is_single_sentence(text):
    # Regular expression to match sentence-ending punctuation
    sentence_endings = re.compile(r'[.!?]')
    
    # Find all sentence-ending punctuation in the text
    matches = sentence_endings.findall(text)
    
    # Check if there is exactly one sentence-ending punctuation
    if len(matches) == 1:
        return True
    else:
        return False

for idx, data in tqdm(enumerate(dataset_abstractive['test'])):
    if not is_single_sentence(data['summary']):
        print(idx)
        print(data['summary'])

687it [00:00, 3450.10it/s]

34
The British government approved Friday the extradition of WikiLeaks founder Julian Assange to the U.S., where he is wanted on charges under the Espionage Act.
37
Russia’s state-owned energy company Gazprom says a key pipeline conveying natural gas to Europe will shut down for three days at the end of this month to undergo “routine maintenance.” In a statement posted online on Friday, Gazprom said that the only operational turbine at a key compressor station along the Nord Stream 1 pipeline, which links Western Russia and Germany, will shut down for routine maintenance from August 31 to September 2.
51
The U.S. government this week is holding its first onshore oil and gas drilling lease auctions since President Joe Biden took office after a federal court blocked the administration’s attempt to suspend such sales because of climate change worries.
59
The All Progressives Grand Alliance has stated that it will not support Mr. Peter Obi, the presidential candidate for the Labour Party, 

1371it [00:00, 3358.32it/s]

735
The Justice Department said Monday that Steve Bannon should serve six months in prison and pay a $200,000 fine for defying a congressional subpoena from the House committee investigating the Jan. 6 insurrection at the U.S.
746
A New Jersey man was sentenced to three years in prison Monday after he admitted to posing as a former Patriots player in an effort to buy and sell Super Bowl rings.S.
769
Fox News Channel is airing the Jan. 6 committee hearings when they occur in daytime hours and a striking number of the network’s viewers have made clear they’d rather be doing something else.
772
Former Trump White House adviser Peter Navarro pleaded not guilty to contempt of Congress charges on Friday over his refusal to comply with subpoenas from the House select committee investigating the Jan. 6 Capitol riot.
775
Pennsylvania's Republican nominee for governor, Doug Mastriano, has offered to sit for an interview with the congressional committee investigating the Jan. 6 attack on the U.S.

2060it [00:00, 3384.13it/s]

...

2910
D.H. Peligro, drummer for the Dead Kennedys and a former member of Red Hot Chili Peppers has died following an accidental fall, aged 63.
2920
A white man accused of killing 10 Black people in a racist attack at a Buffalo, N.Y., supermarket has been charged with federal hate crimes and could face the death penalty.
2929
The former CEO of failed cryptocurrency firm FTX, Sam Bankman-Fried, has been arrested in the Bahamas at the request of the U.S. government, the U.S. and Bahamian authorities said Monday.
2939
The much-awaited and anticipated film 777 Charlie featuring Rakshit Shetty and a dog has hit the theatres today in Kannada, Hindi, Telugu, Tamil, and Malayalam. The film has stood up to all the expectations as it is getting a positive response from the audience.
2943
Ouachita Baptist defensive lineman Clark Yarbrough died Sunday after collapsing suddenly, the school said in a social media post. The school did not provide more information on the cause of death.
2962
Capitol Pol

4313it [00:01, 3715.70it/s]

3668
The Federal Aviation Administration on Wednesday proposed a new rule in order to cut greenhouse gas emissions from new aircraft manufactured in the U.S. after Jan. 1, 2028.
3680
A federal grand jury has indicted Peter Navarro, a former Trump White House adviser, on two counts of criminal contempt of Congress for defying subpoenas from the House Jan. 6 select committee.
3696
Congress president Sonia Gandhi Saturday termed ''directionless'' the Centre's 'Agnipath' scheme for recruitment in the armed forces, and said her party will work for its withdrawal. She also appealed to the youths protesting against the scheme to adopt peaceful and non-violent means to fight for their demands.
3727
TikTok is denying claims that a hacking group has breached an Alibaba cloud database containing 2.05 billion records that include data on TikTok and WeChat users.
3734
After no winner on Tuesday, the Mega Millions jackpot has surged to $1.025 billion, the third-largest prize in the game's history an

4947it [00:01, 3567.04it/s]

4446
India’s market regulator fined Reliance and two of its officers on Monday for not properly disclosing Facebook’s $5.7 billion investment into Jio Platforms in April 2020.
4462
The Biden administration lifted Trump-era flight restrictions that prevented U.S. airlines and chartered flights from going to other cities in Cuba besides Havana on Wednesday.
4469
Emergency services were first called to Babbs Mill Park in the Kingshurst area of Solihull, West Midlands, near Birmingham at 2.36pm on Sunday, where it was reported four children had been playing on the ice and fallen through into the lake.
4486
Uvalde PD released a brief statement on their Facebook page acknowledging a shooting that occurred around 5:30 p.m. in the area of Uvalde Memorial Park that has left multiple people injured.
4519
Odell Beckham Jr., an NFL free agent, was removed by authorities from an aircraft before takeoff at Miami International Airport after officials said he failed to respond to requests to buckle hi